In [1]:
!export CUDA_VISIBLE_DEVICES=0

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())


0


In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.device_count()

1

In [4]:
torch.cuda.current_device()

0

In [5]:
# cudaSetDevice(0)
#dataset transcript要在dataset資好夾調
#cuda 在basesolver init調
#dataset要在data.py設定

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
import torch
import argparse
import numpy as np
import pandas as pd

dfs = []
alldf =pd.DataFrame()

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', './config/cv11Lu_asr_lstm4atthead_allvocab-biclass-test-5-1.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'ckpt/')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', None)
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',8)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', True)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 9)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', False)
    setattr(paras, 'binaryClass', True)
#     force_cudnn_initialization()

    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            from test_binaryclass import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        elif paras.binaryClass:
            assert paras.load is not None
            from train_binaryclass import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))
    for times in range (6,15):
        dfs = []
        alldf =pd.DataFrame()
        for idx in range(5):
            paras.config = f'./config/cv11Lu_asr_lstm4atthead_allvocab-biclass-test-5-{idx+1}.yaml'
            config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)
            config['core']['ckpt'] = f'/mnt/usb/jason3/LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-{idx+1}_sd0/latest{times}knew_oldASR_4LSTM.pth'
            solver = Solver(config, paras, mode)

            solver.load_data()
        #     solver.print_model()
            solver.set_model()
            names, hyps, txts, ans = solver.exec()
            dfs.append(pd.DataFrame(
                {'name': names,
                 'hyps': hyps,
                 'truth': txts,
                 'ifcorrect': ans
                }))
            print(dfs[idx])
            print(names)
            print(hyps)
            print(txts)
            print(ans)
            del solver
        for idx, df in enumerate(dfs):
            alldf = pd.concat([alldf, df])#
            df.to_csv(f'./biclass_result/5folds{times}knew_oldASR_4LSTM{idx+1}.csv', index=False)
        alldf.to_csv(f'./biclass_result/5folds{times}knew_oldASR_4LSTM.csv', index=False) 
#             df.to_csv(f'./biclass_result/5folds{times}k_oldASR_4LSTM{idx+1}.csv', index=False)
#         alldf.to_csv(f'./biclass_result/5folds{times}k_oldASR_4LSTM.csv', index=False) 


0


In [2]:
main()


Using test mode

[INFO] Evaluating result of tr. config @ ../LAS_Mandarin_PyTorch-master/config/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1.yaml
[INFO] Evaluating result of tr. ckpt @ /mnt/usb/jason3/LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1_sd0/latest6knew_oldASR_4LSTM.pth
Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTT5-1']
Mozillacv11Dataset CTT5-1 found wav data: 21
text len: 21
remove None, then wav data: 18, text len: 18
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTTtrain']
Mozillacv11Dataset CTTtrain found wav data: 73
text len: 73
remove None, then wav data: 64, text len: 64
[INFO] Data spec. | Corpus = vag (from /home/jupyter-jason5/data_process)                                  
[INFO]            | Train sets = ['CTTtrain']	| N

/home/jupyter-jason5/LAS_Mandarin_PyTorch-master/core/module.py:51: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  feat_len = feat_len // 4


['54CTT']   0.13704632222652435   0.0   True
['78CTT']   0.054690372198820114   0.0   True
['20CTT']   0.6853271722793579   1.0   True
['74CTT']   0.013111024163663387   0.0   True
['67CTT']   0.5641341805458069   1.0   True
['58CTT']   0.9761502742767334   1.0   True
['68CTT']   0.05795379728078842   0.0   True
['19CTT']   0.28050145506858826   1.0   False
['40CTT']   0.11919212341308594   1.0   False
['94CTT']   0.020618757233023643   0.0   True
['93CTT']   0.058666255325078964   0.0   True
['92CTT']   0.015687281265854836   0.0   True
['71CTT']   0.030425501987338066   0.0   True
['81CTT']   0.3643079996109009   0.0   True
['28CTT']   0.9494155049324036   1.0   True
['47CTT']   0.04345104098320007   0.0   True
['77CTT']   0.0829705223441124   0.0   True
['63CTT']   0.5992387533187866   1.0   True
[INFO] All done !                                                                                          
     name      hyps  truth  ifcorrect
0   54CTT  0.137046    0.0       True
1   7

/home/jupyter-jason5/LAS_Mandarin_PyTorch-master/core/module.py:51: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  feat_len = feat_len // 4


['13CTT']   0.3065629303455353   0.0   True
['51CTT']   0.03157472610473633   0.0   True
['14CTT']   0.001190047711133957   0.0   True
['85CTT']   0.00023728706582915038   0.0   True
['35CTT']   0.8185507655143738   1.0   True
['65CTT']   0.19076696038246155   1.0   False
['22CTT']   0.8532559871673584   1.0   True
['95CTT']   0.06802699714899063   0.0   True
['9CTT']   0.9772053360939026   1.0   True
['2CTT']   0.9204306602478027   0.0   False
['43CTT']   0.048516906797885895   0.0   True
['75CTT']   0.0009736749925650656   0.0   True
['89CTT']   0.05251188948750496   0.0   True
['4CTT']   0.9712603688240051   1.0   True
['37CTT']   0.13464386761188507   1.0   False
['29CTT']   0.10685863345861435   0.0   True
['90CTT']   0.0004179258830845356   0.0   True
['87CTT']   0.09379501640796661   0.0   True
[INFO] All done !                                                                                          
     name      hyps  truth  ifcorrect
0   13CTT  0.306563    0.0       True
1  

...

['3CTT']   1.2343026156713677e-08   0.0   True
['57CTT']   0.8919702172279358   1.0   True
['1CTT']   0.3485660254955292   0.0   True
['8CTT']   0.9942586421966553   1.0   True
['33CTT']   0.9999990463256836   1.0   True
['100CTT']   0.0008990865317173302   0.0   True
['39CTT']   0.9983662962913513   1.0   True
['70CTT']   0.003781316801905632   0.0   True
['83CTT']   0.00022882914345245808   0.0   True
['42CTT']   0.008665976114571095   0.0   True
['69CTT']   4.409843313624151e-05   0.0   True
['88CTT']   1.975150709654372e-08   0.0   True
['5CTT']   0.9994251728057861   1.0   True
['30CTT']   0.008441435173153877   0.0   True
['31CTT']   0.9999998807907104   1.0   True
['80CTT']   0.003083534073084593   0.0   True
['98CTT']   0.0005420196102932096   0.0   True
[INFO] All done !                                                                                          
      name          hyps  truth  ifcorrect
0     3CTT  1.234303e-08    0.0       True
1    57CTT  8.919702e-01    1.0  

/home/jupyter-jason5/LAS_Mandarin_PyTorch-master/core/module.py:51: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  feat_len = feat_len // 4


['44CTT']   0.0010328615317121148   0.0   True
['21CTT']   0.018148496747016907   1.0   False
['16CTT']   0.9999951124191284   1.0   True
['32CTT']   0.9985778331756592   1.0   True
['79CTT']   0.0017005426343530416   0.0   True
['12CTT']   0.0018097114516422153   1.0   False
['96CTT']   0.9999994039535522   0.0   False
['18CTT']   0.9973074197769165   1.0   True
['7CTT']   0.02201024256646633   0.0   True
['84CTT']   0.0003689132281579077   0.0   True
['91CTT']   0.0014997156104072928   0.0   True
['55CTT']   0.000987912411801517   0.0   True
['24CTT']   0.9996194839477539   1.0   True
['6CTT']   0.0307965949177742   0.0   True
['97CTT']   0.004732562694698572   0.0   True
['64CTT']   0.9982661604881287   1.0   True
['56CTT']   0.00015122450713533908   0.0   True
['52CTT']   3.3993048873526277e-06   0.0   True
[INFO] All done !                                                                                          
     name      hyps  truth  ifcorrect
0   44CTT  0.001033    0.0     

/home/jupyter-jason5/LAS_Mandarin_PyTorch-master/core/module.py:51: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  feat_len = feat_len // 4


['53CTT']   0.0004090078000444919   0.0   True
['99CTT']   1.7165551753350883e-06   0.0   True
['38CTT']   1.0   1.0   True
['45CTT']   0.8765639066696167   1.0   True
['34CTT']   0.0006700228550471365   0.0   True
['10CTT']   0.0032459080684930086   1.0   False
['86CTT']   2.8250398827367462e-05   0.0   True
['50CTT']   0.029757477343082428   0.0   True
['15CTT']   0.00046185299288481474   0.0   True
['17CTT']   0.9999852180480957   1.0   True
['82CTT']   2.0891920371468586e-07   0.0   True
['72CTT']   0.9458289742469788   0.0   False
['46CTT']   0.0005023537087254226   0.0   True
['73CTT']   1.725470974633936e-05   0.0   True
['11CTT']   0.9998639822006226   1.0   True
['25CTT']   0.9878660440444946   1.0   True
['76CTT']   0.0001696003309916705   0.0   True
[INFO] All done !                                                                                          
     name          hyps  truth  ifcorrect
0   53CTT  4.090078e-04    0.0       True
1   99CTT  1.716555e-06    0.0       

In [3]:
for idx, df in enumerate(dfs):
    alldf = pd.concat([alldf, df])#
    df.to_csv(f'./biclass_result/5folds12knew_4LSTM{idx+1}.csv', index=False)
alldf.to_csv(f'./biclass_result/5folds12knew_4LSTM.csv', index=False)    

In [2]:
main()


Using test mode

[INFO] Evaluating result of tr. config @ ../LAS_Mandarin_PyTorch-master/config/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1.yaml
Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTTtest']
Mozillacv11Dataset CTTtest found wav data: 21
text len: 21
remove None, then wav data: 21, text len: 21
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTTtrain']
Mozillacv11Dataset CTTtrain found wav data: 73
text len: 73
remove None, then wav data: 73, text len: 73
[INFO] Data spec. | Corpus = vag (from /home/jupyter-jason5/data_process)                                  
[INFO]            | Train sets = ['CTTtrain']	| Number of utts = 73                                        
[INFO]            | Dev sets = ['CTTtest']	| Number of utts = 21                      

In [3]:
from sklearn.metrics import classification_report, confusion_matrix
y_true = [0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0] 
y_pred = [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
target_names = ['non-patient', 'patient']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

 non-patient       0.92      0.80      0.86        15
     patient       0.62      0.83      0.71         6

    accuracy                           0.81        21
   macro avg       0.77      0.82      0.79        21
weighted avg       0.84      0.81      0.82        21



In [6]:
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
print('sensitivity: ',tp / (fn+tp))
print('specificity: ',tn / (fp+tn))


sensitivuty:  0.8333333333333334
specificity:  0.8


In [10]:
import numpy as np
from sklearn import metrics
y = np.array([0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0])
pred = np.array([0.0005, 0.1006, 0.8517, 0.9374, 0.9875, 0.2795, 0.0041, 0.0005, 0.0005, 0.2542, 0.0002, 0.0014, 0.0029, 0.9852, 0.0029, 3.6719e-05, 0.9708, 0.0003, 1.2828e-05, 0.9127, 0.0006])
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
metrics.auc(fpr, tpr)

0.7692307692307693

In [11]:
!python eval.py --file ~/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv


============  Result of /home/jupyter-jason3/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv ============
 -----------------------------------------------------------------------
| Statics		|  Truth	|  Prediction	| Abs. Diff.	|
 -----------------------------------------------------------------------
| Avg. # of chars	|  8.01	|  8.10	|  0.16		|
| Avg. # of words	|  1.00	|  1.00	|  0.00		|
 -----------------------------------------------------------------------
 ---------------------------------------------------------------
| Error Rate (%)| Mean		| Std.		| Min./Max.	|
 ---------------------------------------------------------------
| Character	| 7.9479	| 21.68		| 0.00/240.00	|
| Word		| 18.1725	| 38.56		| 0.00/100.00	|
 ---------------------------------------------------------------
Note : If the text unit is phoneme, WER = PER and CER is meaningless.

